## Inspect VCC functions

Since Houdini 16.5 I no longer need to use any of it to generate
completions and reliable signatures. Still, may be useful in future.

In [1]:
import os.path as op
import random
import copy
import json
import yaml
from subprocess import run, PIPE

HFS = 'C:/Program Files/Side Effects Software/Houdini 16.5.268'

assert op.exists(HFS)

### Fetch data from VCC

In [2]:
vcc_exe = op.join(HFS, 'bin/vcc')
proc = run([vcc_exe, '--list-context-json'],
           shell=True, stdout=PIPE)

# Load VEX contexts list from VCC.
contexts = json.loads(proc.stdout)

# Collect all VEX functions from all contexts.
vcc = {}
for con in contexts:
    proc = run([vcc_exe, '--list-context-json', con],
               shell=True, stdout=PIPE)
    funs = json.loads(proc.stdout)['functions']

    # Include context signatures.
    for f, sigs in funs.items():
        for s in sigs:
            s['contexts'] = [con]

    # Add unique functions and signatures to common dictionary.
    for f in funs:
        # Function not exists.
        if f not in vcc:
            vcc[f] = {'sigs': funs[f]}
            continue

        # Function exists. Search for unique signatures.
        for con_s in funs[f]:
            for s in vcc[f]['sigs']:
                if (con_s['args'], con_s['return']) == (s['args'], s['return']):
                    s['contexts'].append(con)
                    break  # Suppress for-else clause.
            # Non-unique not found (signature is unique).
            else:
                vcc[f]['sigs'].append(con_s)


# Debug VCC output.
rand_fun = random.choice(list(vcc))
print(f'''
Total functions: {len(vcc)}
Total signatures: {sum(len(vcc[fun]['sigs']) for fun in vcc)}

Random function: {rand_fun}
Signatures: {len(vcc[rand_fun]['sigs'])}

{yaml.dump(vcc[rand_fun]['sigs'], width=200)}
''')


Total functions: 741
Total signatures: 3634

Random function: sample_direction_uniform
Signatures: 1

- args: [const vector2]
  contexts: [surface, displace, light, shadow, fog, chop, pop, sop, cop2, image3d, cvex]
  external: true
  return: vector




### Cleanup result

In [3]:
vcc_export = copy.deepcopy(vcc)

for name, fun in vcc_export.items():
    for sig in fun['sigs']:
        # What is it?
        if 'external' in sig:
            del sig['external']
            
        for i in range(len(sig['args'])):
            if 'const' in sig['args'][i]:
                sig['args'][i] = sig['args'][i].replace('const ', '')
            else:
                sig['args'][i] = '&' + sig['args'][i].replace('export ', '')

        if 'variadic' in sig or 'variadic_pair' in sig:
            sig['args'].append('...')
        if 'variadic' in sig:
            del sig['variadic']
        if 'variadic_pair' in sig:
            del sig['variadic_pair']
            
        # Remove contexts from all-contexts functions.
        if set(sig['contexts']) == set(contexts):
            del sig['contexts']
            
        if not sig['args']:
            del sig['args']
            
        if sig['return'] == 'void':
            del sig['return']

    # Discard deprecated signatures.
    vcc_export[name] = {'sigs': [s for s in fun['sigs'] if 'deprecated' not in s]}


# Debug result.
print(f'''Clean version: {rand_fun}
Signatures: {len(vcc_export[rand_fun]['sigs'])}
{yaml.dump(vcc_export[rand_fun]['sigs'], width=200)}
''')


# Export to YAML.
with open('vcc.yml', 'w') as f:
    yaml.dump(vcc_export, f, width=256)

Clean version: sample_direction_uniform
Signatures: 1
- args: [vector2]
  return: vector




### Extra

In [7]:
# Inspect variadic functions of all sorts.

variadic = set()
varpair = set()
both = set()

for name, fun in vcc.items():
    for sig in fun['sigs']:
        if 'variadic' in sig:
            variadic.add(name)
        if 'variadic_pair' in sig:
            varpair.add(name)

print(f'''
Variadic:
{' '.join(sorted(variadic - varpair))}

Variadic pair:
{' '.join(sorted(varpair - variadic))}

Both:
{' '.join(sorted(variadic & varpair))}
''')


Variadic:
array avg ckspline concat cspline error fuzzy_and fuzzy_nand fuzzy_nor fuzzy_nxor fuzzy_or fuzzy_xor getsmoothP haslight kspline lkspline lspline max min newsampler nrandom print_once printf spline sprintf texprintf warning

Variadic pair:
addvariablename pcconvex pcfarthest pcnumfound pcsize solid_angle

Both:
Du Dv Dw addattribute albedo ambient area ashikhmin binput blinn cinput colormap computenormal cone cvex_bsdf diffuse environment eval_bsdf expand_udim fastshadow file_stat filtershadow filterstep finput getderiv getlights gradient hair henyeygreenstein intersect intersect_lights irradiance isotropic matchvex_blinn matchvex_specular nextsample ninput occlusion pathtrace pcexport pcfilter pcopen pcopenlod pcwrite phong phonglobe photonmap ptexture rawcolormap rayhittest reflectlight refractlight resolvemissedray sample_bsdf sample_geometry sample_light shadow_light shadowmap specular sssapprox texture texture3d texture3dBox trace translucent volume wireblinn wirediffus

In [6]:
# Inspect external functions.

external = set()
nonexternal = set()

for name, fun in vcc.items():
    for sig in fun['sigs']:
        if 'external' in sig:
            external.add(name)
        else:
            nonexternal.add(name)

print(f'''
External-only:
{' '.join(sorted(external - nonexternal))}

Non-external-only:
{' '.join(sorted(nonexternal - external))}

Both:
{' '.join(sorted(nonexternal & external))}
''')


External-only:
adddetailattrib addpointattrib addprimattrib addvertexattrib agentclipcatalog agentclipnames agentcliptimes agentcliptransformgroups agentclipweights agentcollisionlayer agentcurrentlayer agentlayers agenttransformgroups agenttransformnames append assign attrib attribclass blackbody chadd chinputlimits chnames chremove chremoveattr chrename chsetlength chsetrate chsetstart clamp cross degrees detailattribsize detailattribtype detailattribtypeinfo detailintrinsic distance distance2 dot efit endswith filamentsample fit fit01 fit10 fit11 fuzzify fuzzy_defuzz_centroid geoself geounwrap getattrib getbbox getbbox_center getbbox_max getbbox_min getbbox_size getpointbbox_center getpointbbox_max getpointbbox_min getpointbbox_size hasdetailattrib haspointattrib hasprimattrib hasvertexattrib hedge_dstpoint hedge_next hedge_postdstpoint hedge_presrcpoint hedge_prev hedge_srcpoint ident idtopoint idtoprim ingroup isalpha isdigit isvalidindex length length2 limit_sample_space luminan

WTF is this?